In [3]:
import numpy as np
import pandas as pd
import sys
if '../source' not in sys.path: sys.path.insert(0, '../source')
from utils import *
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import skopt
from skopt import BayesSearchCV
from skopt.space import Integer, Real

skopt.__version__

'0.9.0'

## Data import 
![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

In [4]:
# Read the data, convert the 'Time' column to datetime format, set it as the index, and display the first few rows.
df = pd.read_csv('../saves/meteo_p_dc_22_init_impute.csv')
df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S')
df.set_index('Time', inplace=True)
df.head()

,GTI,GHI,DNI,DHI,Air_Temp,RH,Pressure,Wind_speed,Wind_dir,Wind_gust,Rain,P_DC,Imputation
Time,,,,,,,,,,,,,
2022-01-01 00:00:00,0.0,0.0,0.0,0.0,10.170313,77.0750,969.5,0.0,0.0,0.0,0.0,0.0,NaN
2022-01-01 00:10:00,0.0,0.0,0.0,0.0,10.050000,78.3500,969.5,0.0,0.0,0.0,0.0,0.0,NaN
2022-01-01 00:20:00,0.0,0.0,0.0,0.0,9.731250,79.0375,969.5,0.0,0.0,0.0,0.0,0.0,NaN
2022-01-01 00:30:00,0.0,0.0,0.0,0.0,9.560938,80.2750,969.5,0.0,0.0,0.0,0.0,0.0,NaN
2022-01-01 00:40:00,0.0,0.0,0.0,0.0,9.790625,79.6625,969.5,0.0,0.0,0.0,0.0,0.0,NaN


## Data preparation 
![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

In [55]:
df_lags = df[['GTI', 'GHI', 'DNI', 'DHI', 'Air_Temp', 'RH', 'P_DC']].copy(deep=True)

# Define the lag order as 12 * 6, (12 hours assuming data is sampled at 10-minute intervals).
lag_order = 12 * 6  # 12 hours
# Create lag features for the 'P_DC' column by shifting it forward in time up to 12 hours
for lag in range(1, lag_order + 1):
    df_lags[f'P_DC_Lag{lag}'] = df_lags['P_DC'].shift(lag)
# remove rows with missing values.
df_lags.dropna(inplace=True) 

In [56]:
X, y = df_lags.drop(columns=['P_DC']).values, df_lags['P_DC'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True)

## Training and Bayesian Hyperparameter Optimization 
![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

In [57]:
import warnings
warnings.filterwarnings('ignore')
import multiprocessing
multiprocessing.cpu_count()

16

In [58]:
# Create an XGBoost regressor with the squared error as the objective function and GPU acceleration enabled.
estimator = XGBRegressor(objective='reg:squarederror', tree_method="gpu_hist", gpu_id=0)

# Define regressor fitting parameters
fit_params = {
    'early_stopping_rounds': 10,
    'eval_set':[(X_val, y_val)],
    'verbose': True,
}

# Define a search space for hyperparameter optimization.
search_space = {
    'max_depth': Integer(3, 6),
    'n_estimators': Integer(200, 1500),
    'learning_rate': Real(0.01, 0.3, 'log-uniform'),
    'gamma': Real(1e-9, 0.5, 'log-uniform'),
}

# Create a Bayesian hyperparameter search using BayesSearchCV with the XGBoost estimator, the defined search space, cross-validation with 3 folds, 
# using negative mean squared error as the scoring metric, and parallel processing with 3 jobs.

opt = BayesSearchCV(
    estimator=estimator,
    search_spaces=search_space,
    fit_params=fit_params,
    cv=3,
    scoring="neg_mean_squared_error",
    random_state=42,
    n_iter=50,
    n_points=2,
    n_jobs=3,
    verbose=0,
)

# Lunching the search (n_iter iterations) to find the best hyperparameters using the training data.
opt.fit(X_train, y_train)

C:\Users\oussa\miniconda3\envs\gep-intern\Lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning:

The objective has been evaluated at this point before.

C:\Users\oussa\miniconda3\envs\gep-intern\Lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning:

The objective has been evaluated at this point before.

C:\Users\oussa\miniconda3\envs\gep-intern\Lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning:

The objective has been evaluated at this point before.

C:\Users\oussa\miniconda3\envs\gep-intern\Lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning:

The objective has been evaluated at this point before.

C:\Users\oussa\miniconda3\envs\gep-intern\Lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning:

The objective has been evaluated at this point before.

C:\Users\oussa\miniconda3\envs\gep-intern\Lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning:

The objective has been evaluated at this point before.

C:\Users\oussa\m

BayesSearchCV(cv=3,
              estimator=XGBRegressor(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None, gpu_id=0,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max...
              scoring='neg_mean_squared_error',
              search_spaces={'gamma': Real(low=1e-09, high=0.5, prior='log-uniform', transform='normalize'),
                             'learning_rate': Real(low=0.01, high=0.3, prior='log-uniform', transform='normalize'),
                             'max_depth': Integer(low=3, high=6, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=200, high=1500, prior='uniform', transform='normalize')})

In [259]:
opt.best_params_

OrderedDict([('gamma', 0.49999999999999994),
             ('learning_rate', 0.01),
             ('max_depth', 6),
             ('n_estimators', 674)])

In [260]:
# Generate a bar chart of the top 20 most important features and their coefficients

names = df_lags.columns.to_list()
names.remove('P_DC')
most_important_features = sorted(zip(names, opt.best_estimator_.feature_importances_), key=lambda x: x[1], reverse=True)
names = [name for name, _ in most_important_features]
coefficients = [coefficient for _, coefficient in most_important_features]

fig = go.Figure([go.Bar(x=names[:20], y=coefficients[:20])])
fig.show()

In [70]:
import json

# Save the best parameters
with open('../saves/opt_params_31_08_23.json', 'w') as f:
    json.dump(dict(opt.best_params_), f)

# Save the best model 
opt.best_estimator_.save_model('../saves/opt_model_31_08_23.json')

## Evaluation
![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

In [71]:
# Perform prediction using the best model
y_train_pred = opt.best_estimator_.predict(X_train)
y_test_pred = opt.best_estimator_.predict(X_test)

# evaluation metrics
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Train Results:")
print(f"Mean Squared Error (MSE): {train_mse:.3f}")
print(f"Root Mean Squared Error (RMSE): {np.sqrt(train_mse):.3f}")
print(f"Mean Absolute Error (MAE): {train_mae:.3f}")
print(f"R-squared (R2): {train_r2:.3f}")

print("\nTest Results:")
print(f"Mean Squared Error (MSE): {test_mse:.3f}")
print(f"Root Mean Squared Error (RMSE): {np.sqrt(test_mse):.3f}")
print(f"Mean Absolute Error (MAE): {test_mae:.3f}")
print(f"R-squared (R2): {test_r2:.3f}")

Train Results:
Mean Squared Error (MSE): 46993.178
Root Mean Squared Error (RMSE): 216.779
Mean Absolute Error (MAE): 78.575
R-squared (R2): 0.993

Test Results:
Mean Squared Error (MSE): 58864.128
Root Mean Squared Error (RMSE): 242.619
Mean Absolute Error (MAE): 90.375
R-squared (R2): 0.990


In [68]:
def multi_step_forecast(model, X_meteo, X_p_dc, num_forecast_steps=1):
    
    # Initialize an array to store the forecasts
    forecasts = []

    current_p_dc = X_p_dc

    # Perform multi-step forecasting
    for i in range(min(num_forecast_steps, X_meteo.shape[0])):
        
        # Prepare the input by stacking meteo data and power data
        current_features = np.hstack((X_meteo[i], current_p_dc))
        
        # Predict the next step
        next_step_forecast = model.predict(current_features.reshape(1, -1))

        # Append the forecast to the list
        forecasts.append(next_step_forecast[0])

        # Update the current features for the next iteration
        current_p_dc = np.roll(current_p_dc, 1)
        current_p_dc[0] = next_step_forecast

    return forecasts

In [869]:
gap_lengths = [24*i for i in range(1, 7)]
results = {}
evaluation_metrics = {
    'mse' : mean_squared_error,
    'rmse': lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False),
    'mae' : mean_absolute_error,
    'r2' : r2_score
}

for gap_len in gap_lengths:
    results[gap_len] = {metric:[] for metric in evaluation_metrics.keys()}

    for i in range(df_lags.shape[0] // lag_order):
        X_meteo = df_lags.iloc[i*lag_order:(i+1)*lag_order, :6].values
        X_p_dc = df_lags.iloc[i*lag_order, 7:].values
        y_true = df_lags.iloc[i*lag_order:(i+1)*lag_order, 6].values
    
        forcasts = multi_step_forecast(opt, X_meteo, X_p_dc, lag_order)
        
        for metric, metric_func in evaluation_metrics.items():
            results[gap_len][metric].append(metric_func(y_true[:gap_len], forcasts[:gap_len]))
    
    results[gap_len] = {metric:np.mean(mesures) for metric, mesures in results[gap_len].items()}

In [871]:
import pprint

pprint.pprint(results)

{24: {'mae': 171.2927,
      'mse': 121979.04,
      'r2': -7396.130906123588,
      'rmse': 233.80052},
 48: {'mae': 190.40318,
      'mse': 234548.22,
      'r2': -195.23714264502644,
      'rmse': 280.40802},
 72: {'mae': 205.51892,
      'mse': 288678.03,
      'r2': -188.6925800951791,
      'rmse': 316.73227},
 96: {'mae': 205.51892,
      'mse': 288678.03,
      'r2': -188.6925800951791,
      'rmse': 316.73227},
 120: {'mae': 205.51892,
       'mse': 288678.03,
       'r2': -188.6925800951791,
       'rmse': 316.73227},
 144: {'mae': 205.51892,
       'mse': 288678.03,
       'r2': -188.6925800951791,
       'rmse': 316.73227}}


## Visualization of test samples
![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

In [262]:
# Load XGB regressor
xgb_model = XGBRegressor()
xgb_model.load_model('../saves/opt_model_31_08_23.json')

In [283]:
fig = make_subplots(rows=3, cols=2, shared_yaxes=True)
k = 15  # change k for other samples

for i in range(6):
    X_meteo = X_test[(i+k)*lag_order+i*k*3:(i+1+k)*lag_order+i*k*3, :6]
    X_p_dc = X_test[(i+k)*lag_order+i*k*3, 6:]
    y_true = y_test[(i+k)*lag_order+i*k*3:(i+1+k)*lag_order+i*k*3]
    
    forecasts = multi_step_forecast(xgb_model, X_meteo, X_p_dc, lag_order)

    X_p_dc = np.flip(X_p_dc)
    
    fig.add_trace(go.Scatter(y=np.hstack((X_p_dc, y_true)),
                    mode='lines', line=dict(color='LightSkyBlue', width=2), name='Real'), row=1+i//2, col=1+i%2)
    fig.add_trace(go.Scatter(y=np.hstack((X_p_dc, forecasts)),
                    mode='lines', line=dict(color='red', width=2), name='Forecasts'), row=1+i//2, col=1+i%2)
    fig.add_trace(go.Scatter(y=X_test[(i+k-1)*lag_order+i*k*3:(i+1+k)*lag_order+i*k*3, 0],
                    mode='lines', marker=dict(color='green', size=1), line=dict(width=1), name='GTI'), row=1+i//2, col=1+i%2)

fig.update_layout(height=700, width=1100, showlegend=False,
                  title_text="Frocasts on test samples")
fig.show()

In [264]:
df.isna().sum()

GTI               0
GHI               0
DNI               0
DHI               0
Air_Temp          0
RH                0
Pressure          0
Wind_speed        0
Wind_dir          0
Wind_gust         0
Rain              0
P_DC            665
Imputation    48625
dtype: int64

## Filling gaps in the original data
![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

In [266]:
# Extract existing gaps
gaps = find_gaps(df.P_DC)
print('%d remaining gaps' % len(gaps))
gaps_len = [(gap[1] - gap[0]).seconds // 60 // 10 // 6 for gap in gaps]
print('Longest remaining gap %dh' % max(gaps_len))

18 remaining gaps
Longest remaining gap 13h


In [268]:
df_imputed = df.copy(deep=True)
df_imputed['P_DC_XGB'] = df_imputed['P_DC'].copy(deep=True)
df_imputed['Imputation_XGB'] = df_imputed['Imputation'].copy(deep=True)

In [269]:
# Impute remaining gaps
for gap in gaps:
    # Extract the meteo data that corresponds to the 'gap' time range
    meteo_values = df_imputed[['GTI', 'GHI', 'DNI', 'DHI', 'Air_Temp', 'RH']][gap[0]:gap[1]].values
    # Extract the power lagged values for the window before the start of the current 'gap'
    lagged_values = df_imputed.loc[df_imputed.index < gap[0], 'P_DC_XGB'].tail(lag_order).values
    forecast_step = meteo_values.shape[0]
    # Perform a multi-step forecast
    forecasts = multi_step_forecast(xgb_model, meteo_values, lagged_values, forecast_step)
    df_imputed.loc[gap[0]:gap[1], 'P_DC_XGB'] = forecasts
    df_imputed.loc[gap[0]:gap[1], 'Imputation_XGB'] = 2.

In [270]:
df_imputed.isna().sum()

GTI                   0
GHI                   0
DNI                   0
DHI                   0
Air_Temp              0
RH                    0
Pressure              0
Wind_speed            0
Wind_dir              0
Wind_gust             0
Rain                  0
P_DC                665
Imputation        48625
P_DC_XGB              0
Imputation_XGB    47960
dtype: int64

In [274]:
# Number of imputed values by method:  0: Night values, 1: linear interpolation, 2: XGB Regressor
df_imputed.Imputation_XGB.value_counts().sort_index()

Imputation_XGB
0.0    3802
1.0     133
2.0     665
Name: count, dtype: int64

In [101]:
# Save imputed data
df_imputed.to_csv('../saves/df_meteo_p_dc_imputed_xgb.csv')

## Visualization of the results
![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

In [275]:
df_imputed_xgb = pd.read_csv('../saves/df_meteo_p_dc_imputed_xgb.csv')
df_imputed_xgb['Time'] = pd.to_datetime(df_imputed_xgb['Time'], format='%Y-%m-%d %H:%M:%S')
df_imputed_xgb.set_index('Time', inplace=True)
df_imputed_xgb.head()

,GTI,GHI,DNI,DHI,Air_Temp,RH,Pressure,Wind_speed,Wind_dir,Wind_gust,Rain,P_DC,Imputation,P_DC_XGB,Imputation_XGB
Time,,,,,,,,,,,,,,,
2022-01-01 00:00:00,0.0,0.0,0.0,0.0,10.170313,77.0750,969.5,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN
2022-01-01 00:10:00,0.0,0.0,0.0,0.0,10.050000,78.3500,969.5,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN
2022-01-01 00:20:00,0.0,0.0,0.0,0.0,9.731250,79.0375,969.5,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN
2022-01-01 00:30:00,0.0,0.0,0.0,0.0,9.560938,80.2750,969.5,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN
2022-01-01 00:40:00,0.0,0.0,0.0,0.0,9.790625,79.6625,969.5,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN


In [285]:
# Visualize Gap Filling Results

fig = make_subplots(rows=3, cols=2, shared_yaxes=True)

k = 6
for i in range(6):    
    start = gaps[i+k][0] - pd.Timedelta(hours=10)
    end = gaps[i+k][1] + pd.Timedelta(hours=10)
    fig.add_trace(go.Scatter(y=df_imputed.loc[start:end, 'GTI'],
                    mode='lines', line=dict(width=1), marker=dict(color='green', size=2)), row=1+i//2, col=1+i%2)
    fig.add_trace(go.Scatter(y=df_imputed.loc[start:end, 'P_DC_XGB'],
                    mode='lines', marker=dict(color='red')), row=1+i//2, col=1+i%2)
    fig.add_trace(go.Scatter(y=df_imputed.loc[start:end, 'P_DC'],
                    mode='lines', marker=dict(color='LightSkyBlue')), row=1+i//2, col=1+i%2)

fig.update_layout(height=700, width=1100, showlegend=False,
                  title_text="Imputation on test samples")
fig.show()